# Extract audio features 

major features to extract :
- chroma_stft
- chroma_cqt
- chroma_cens
- mfcc
- rms
- spectral_centroid
- spectral_bandwidth
- spectral_contrast
- spectral_rolloff
- tonnetz
- zero_crossing_rate

Total expected dimensions after transformation - 518

In [2]:
# import necessary libraries
import numpy as np 
import pandas as pd 
import librosa 
from glob import glob

In [3]:
#load dataset 
import os
from scipy.io import wavfile
x, sr = librosa.load('./chk_fMale.wav', mono=False)
#data = pd.read_csv('extracted_chorus.csv')
x

array([-0.00144683, -0.00164567, -0.0015117 , ...,  0.00162429,
       -0.00092469,  0.        ], dtype=float32)

In [4]:
#extract F0 using yin
#f0 = librosa.pyin(x, fmin = 0, fmax= 280)
#f0
#print(f0)

In [5]:
import numpy as np

def spectral_properties(y: x, fs: int) -> dict:
    spec = np.abs(np.fft.rfft(y))
    freq = np.fft.rfftfreq(len(y), d=1 / fs)
    spec = np.abs(spec)
    amp = spec / spec.sum()
    mean = (freq * amp).sum()
    sd = np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    amp_cumsum = np.cumsum(amp)
    median = freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    mode = freq[amp.argmax()]
    Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
    Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
    IQR = Q75 - Q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4

    result_d = {
        #'Freq':freq,
        'mean': mean,
        'sd': sd,
        'median': median,
        'mode': mode,
        'Q25': Q25,
        'Q75': Q75,
        'IQR': IQR,
        'skew': skew,
        'kurt': kurt
    }

    return result_d

In [7]:
spectral_properties(x,sr)

{'mean': 2021.4497777140984,
 'sd': 1889.7218762176979,
 'median': 1344.5292513267227,
 'mode': 225.3314163689029,
 'Q25': 474.74998414275746,
 'Q75': 3083.1553798338164,
 'IQR': 2608.405395691059,
 'skew': 5.785845451019724,
 'kurt': 49.53810784227687}

In [6]:
import amfm_decompy.pYAAPT as pYAAPT
import amfm_decompy.basic_tools as basic

signal = basic.SignalObj('f2.wav')
pitch = pYAAPT.yaapt(signal)
pitch.samp_values
average = Average(pitch.samp_values)
print("Average of the list =", round(average, 8))

NameError: name 'Average' is not defined

In [ ]:
#find fundamental frequency and then average of fundamental frequency measured across acoustic signal
#chk formula of IQR from the paper. and then also check the formula of Q25 
#normalize it with RMS technique or another one as well.
#
#Q_) diff between meanfun and meanfreq
# difference between mean frequency and average of fundamental frequency measured across acoustic signal

#Average fundamental frequency 
#during conversation for males ranges from 100 to 150 Hz, whereas for females it ranges from 180 to 250 Hz.


#generate your own .wav formated file

In [ ]:
#Q1 = np.median(x[:10])
  
# Third quartile (Q3)
#Q3 = np.median(x[10:])
  
# Interquartile range (IQR)
#IQR = Q3 - Q1
# Python program to get average of a list
def Average(lst):
    return sum(lst) / len(lst)


In [7]:
#Applying normalization

from sklearn import preprocessing
import numpy as np
x_array = np.array([2608.405395691059, 474.74998414275746,164.78213062])
normalized_arr = preprocessing.normalize([x_array])
normalized_arr

#from sklearn import preprocessing
#import pandas as pd
#import numpy as np

#min_max_Scalar = preprocessing.MinMaxScaler(feature_range(0,2))
#col = [2301.9036685000265,463.74486036204405,443.6923334984471]
#result = min_max_Scalar.fit_transform(col)
#min_max_Scalar_df = pd.DataFrame(result, Colums=col)
# normalize the data attribute
#print("Normalized Data = ", min_max_Scalar_df)

array([[0.98194231, 0.17872111, 0.06203274]])

# import amfm_decompy.basic_tools as basic
import amfm_decompy.pYAAPT as pYAAPT


# load audio
signal = basic.SignalObj('chk_fMale.wav')
filename = 'chk_fMale.wav'


# YAAPT pitches 
pitchY = pYAAPT.yaapt(signal, f0_min=75, f0_max=600)

#pitchY = pYAAPT.yaapt(signal, frame_length=10, tda_frame_length=40, f0_min=60, f0_max=300)
average = np.mean(pitch.samp_values)
print("Average of the list =", round(average, 8))

In [ ]:
def cmvn(vec, variance_normalization=False):
    """ This function is aimed to perform global cepstral mean and
        variance normalization (CMVN) on input feature vector "vec".
        The code assumes that there is one observation per row.

    Args:
        vec (array): input feature matrix
            (size:(num_observation,num_features))
        variance_normalization (bool): If the variance
            normilization should be performed or not.

    Return:
          array: The mean(or mean+variance) normalized feature vector.
    """
    eps = 2**-30
    rows, cols = vec.shape

    # Mean calculation
    norm = np.mean(vec, axis=0)
    norm_vec = np.tile(norm, (rows, 1))

    # Mean subtraction
    mean_subtracted = vec - norm_vec

    # Variance normalization
    if variance_normalization:
        stdev = np.std(mean_subtracted, axis=0)
        stdev_vec = np.tile(stdev, (rows, 1))
        output = mean_subtracted / (stdev_vec + eps)
    else:
        output = mean_subtracted

    return output


# min max

In [ ]:
import numpy as np
data=[[0.98194231, 0.17872111, 0.06203274]]
(data - np.min(data)) / (np.max(data) - np.min(data))

In [45]:
cmvn([2301.9036685000265,463.74486036204405,164.782130])

AttributeError: 'list' object has no attribute 'shape'

In [24]:
#ar = [2301.9036685000265,463.74486036204405,164.782130]
#cmvn(ar)

a = [2301.9036685000265,463.74486036204405,164.782130]
amin, amax = min(a), max(a)
for i, val in enumerate(a):
    a[i] = (val-amin) / (amax-amin)
a

[1.0, 0.12684764380557811, 0.0]

In [21]:
from scipy.stats import skew, kurtosis  # import libs for statistics

In [40]:
columns_name=[]
data=[]

In [41]:
def statistics(list, feature, columns_name, data):
    i = 0 # label to track the number of raw dimensions for each audio feature 
    for ele in list:
        # get the 7 statististics of each raw dimension
       _skew = skew(ele)
        columns_name.append(f'{feature}_kew_{i}')
        min = np.min(ele)
        columns_name.append(f'{feature}_min_{i}')
        max = np.max(ele)
        columns_name.append(f'{feature}_max_{i}')
        std = np.std(ele)
        columns_name.append(f'{feature}_std_{i}')
        mean = np.mean(ele)
        columns_name.append(f'{feature}_mean_{i}')
        median = np.median(ele)
        columns_name.append(f'{feature}_median_{i}')
        _kurtosis = kurtosis(ele)
        columns_name.append(f'{feature}_kurtosis_{i}')

        i += 1
        # append 7 statistics as single row in list 
        data.append(_skew) 
        data.append(min)
        data.append(max) 
        data.append(std) 
        data.append(mean) 
        data.append(median) 
        data.append(_kurtosis)

IndentationError: unexpected indent (<ipython-input-41-92c6f1e786be>, line 6)

In [44]:
data = []
chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
statistics(chroma_stft, 'chroma_stft', columns_name, data) 
chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
statistics(chroma_cqt, 'chroma_cqt', columns_name, data) # stats = 12 dims * 7 stats data

chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
statistics(chroma_cens, 'chroma_cens', columns_name, data) # stats = 12 dims * 7 stats data

mfcc = librosa.feature.mfcc(x, sr) # mfcc with 20 raw dims
statistics(mfcc, 'mfcc', columns_name, data)  # stats = 20 dims * 7 stats data

rms = librosa.feature.rms(x, sr) # rms with 1 raw dims
statistics(rms, 'rms', columns_name, data) # stats = 1 dims * 7 stats data

spectral_centroid = librosa.feature.spectral_centroid(x , sr) # spectral_centroid with 1 raw dims
statistics(spectral_centroid, 'spectral_centroid', columns_name, data) # stats = 1 dims * 7 stats data

spectral_bandwidth = librosa.feature.spectral_bandwidth(x , sr) # spectral_bandwidth with 1 raw dims
statistics(spectral_bandwidth, 'spectral_bandwidth', columns_name, data) # stats = 1 dims * 7 stats data

spectral_contrast = librosa.feature.spectral_contrast(x , sr) # spectral_contrast with 7 raw dims
statistics(spectral_contrast, 'spectral_contrast', columns_name, data) # stats = 7 dims * 7 stats data

spectral_rolloff = librosa.feature.spectral_rolloff(x , sr) # spectral_rolloff with 1 raw dims
statistics(spectral_rolloff, 'spectral_rolloff', columns_name, data) # stats = 1 dims * 7 stats data

tonnetz = librosa.feature.tonnetz(x , sr) # tonnetz with 6 raw dims
statistics(tonnetz, 'tonnetz', columns_name, data) # stats = 6 dims * 7 stats data

zero_crossing_rate = librosa.feature.zero_crossing_rate(x , sr) # zero_crossing_rate with 1 raw dims
statistics(zero_crossing_rate, 'zero_crossing_rate', columns_name, data) # stats = 1 dims * 7 stats data


<ipython-input-44-438cf7b6899f>:2: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
<ipython-input-44-438cf7b6899f>:4: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims


0.8713425397872925
chroma_stft_kew_0
/n
0.9121471047401428
chroma_stft_kew_1
/n
1.5326887369155884
chroma_stft_kew_2
/n
1.6895085573196411
chroma_stft_kew_3
/n
1.5708085298538208
chroma_stft_kew_4
/n
1.6844393014907837
chroma_stft_kew_5
/n
1.4250614643096924
chroma_stft_kew_6
/n
0.8259114623069763
chroma_stft_kew_7
/n
0.4637276530265808
chroma_stft_kew_8
/n
0.44163593649864197
chroma_stft_kew_9
/n
1.0036513805389404
chroma_stft_kew_10
/n
1.1058902740478516
chroma_stft_kew_11
/n
0.9109829664230347
chroma_cqt_kew_0
/n
0.7145799994468689
chroma_cqt_kew_1
/n
2.0452747344970703
chroma_cqt_kew_2
/n
1.5357154607772827
chroma_cqt_kew_3
/n
2.1799561977386475
chroma_cqt_kew_4
/n
2.1898868083953857
chroma_cqt_kew_5
/n
2.168626308441162
chroma_cqt_kew_6
/n
1.104569673538208
chroma_cqt_kew_7
/n
0.4857994318008423
chroma_cqt_kew_8
/n
0.3691681921482086
chroma_cqt_kew_9
/n
1.0146573781967163
chroma_cqt_kew_10
/n
1.0663881301879883
chroma_cqt_kew_11
/n


<ipython-input-44-438cf7b6899f>:7: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims


-0.04470125585794449
chroma_cens_kew_0
/n
0.2333853393793106
chroma_cens_kew_1
/n
1.0595563650131226
chroma_cens_kew_2
/n
0.4773382246494293
chroma_cens_kew_3
/n
0.5848148465156555
chroma_cens_kew_4
/n
1.7364553213119507
chroma_cens_kew_5
/n
1.3805179595947266
chroma_cens_kew_6
/n
0.6110238432884216
chroma_cens_kew_7
/n
-0.2981008291244507
chroma_cens_kew_8
/n
0.38190823793411255
chroma_cens_kew_9
/n
-0.04997636377811432
chroma_cens_kew_10
/n
0.005026209633797407
chroma_cens_kew_11
/n
-0.8354613780975342
mfcc_kew_0
/n
-0.6769257187843323
mfcc_kew_1
/n
-0.10261586308479309
mfcc_kew_2
/n
0.11201474815607071
mfcc_kew_3
/n
0.0904279500246048
mfcc_kew_4
/n
0.20041388273239136
mfcc_kew_5
/n
-0.11802982538938522
mfcc_kew_6
/n
0.39140182733535767
mfcc_kew_7
/n
0.2919740378856659
mfcc_kew_8
/n
-0.1338782012462616
mfcc_kew_9
/n
-0.030171245336532593
mfcc_kew_10
/n
0.24291403591632843
mfcc_kew_11
/n
0.43480074405670166
mfcc_kew_12
/n
0.13350597023963928
mfcc_kew_13
/n
-0.06571871787309647
mfcc_ke

<ipython-input-44-438cf7b6899f>:10: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(x, sr) # mfcc with 20 raw dims
<ipython-input-44-438cf7b6899f>:13: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], S=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = librosa.feature.rms(x, sr) # rms with 1 raw dims
<ipython-input-44-438cf7b6899f>:16: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_centroid = librosa.feature.spectral_centroid(x , sr) # spectral_centroid with 1 raw dims
<ipython-input-44-438cf7b6899f>:19: FutureWarning: 

0.9721230833553794
spectral_contrast_kew_0
/n
0.1105659509114533
spectral_contrast_kew_1
/n
0.022758055599201642
spectral_contrast_kew_2
/n
0.1579044589525145
spectral_contrast_kew_3
/n
0.6448857931096419
spectral_contrast_kew_4
/n
0.7133390491304207
spectral_contrast_kew_5
/n
-1.2876277226194897
spectral_contrast_kew_6
/n
0.18827537148749812
spectral_rolloff_kew_0
/n


<ipython-input-44-438cf7b6899f>:25: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  spectral_rolloff = librosa.feature.spectral_rolloff(x , sr) # spectral_rolloff with 1 raw dims
<ipython-input-44-438cf7b6899f>:28: FutureWarning: Pass y=[ 0.0030202   0.00392768  0.00404555 ...  0.00636663 -0.00016454
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  tonnetz = librosa.feature.tonnetz(x , sr) # tonnetz with 6 raw dims


1.3807690628636815
tonnetz_kew_0
/n
0.8546126353090263
tonnetz_kew_1
/n
0.2542423161420315
tonnetz_kew_2
/n
-0.3462086400024975
tonnetz_kew_3
/n
-0.23217529646366214
tonnetz_kew_4
/n
0.5268168450313346
tonnetz_kew_5
/n
0.06140353911757395
zero_crossing_rate_kew_0
/n


<ipython-input-44-438cf7b6899f>:31: FutureWarning: Pass frame_length=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  zero_crossing_rate = librosa.feature.zero_crossing_rate(x , sr) # zero_crossing_rate with 1 raw dims


In [26]:
columns_name

['chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0',
 'chroma_stft_kew_0']

In [11]:
data = [] # list to contain extracted audio features 

In [16]:
# helper function to get statistics of each dimension data for each audio raw feature 
def statistics(list, feature, columns_name, data):
    i = 0 # label to track the number of raw dimensions for each audio feature 
    for ele in list:
        # get the 7 statististics of each raw dimension
        _skew = skew(ele)
        columns_name.append(f'{feature}_kew_{i}')
        min = np.min(ele)
        columns_name.append(f'{feature}_min_{i}')
        max = np.max(ele)
        columns_name.append(f'{feature}_max_{i}')
        std = np.std(ele)
        columns_name.append(f'{feature}_std_{i}')
        mean = np.mean(ele)
        columns_name.append(f'{feature}_mean_{i}')
        median = np.median(ele)
        columns_name.append(f'{feature}_median_{i}')
        _kurtosis = kurtosis(ele)
        columns_name.append(f'{feature}_kurtosis_{i}')

        i += 1
        # append 7 statistics as single row in list 
        data.append(_skew) 
        data.append(min)
        data.append(max) 
        data.append(std) 
        data.append(mean) 
        data.append(median) 
        data.append(_kurtosis)
    
    return data

In [17]:
# helper function ectract audio feature with statistics 
def extract_features(audio_path, title):

    data = []
    columns_name = ['title']
    data.append(title)
    x , sr = librosa.load(audio_path) #load audio waveform and sample rate

    chroma_stft = librosa.feature.chroma_stft(x, sr) # chroma_stft with 12 raw dims
    statistics(chroma_stft, 'chroma_stft', columns_name, data) # stats = 12 dims * 7 stats data

    chroma_cqt = librosa.feature.chroma_cqt(x, sr) # chroma_cqt with 12 raw dims
    statistics(chroma_cqt, 'chroma_cqt', columns_name, data) # stats = 12 dims * 7 stats data

    chroma_cens = librosa.feature.chroma_cens(x, sr) # chroma_cens with 12 raw dims
    statistics(chroma_cens, 'chroma_cens', columns_name, data) # stats = 12 dims * 7 stats data

    mfcc = librosa.feature.mfcc(x, sr) # mfcc with 20 raw dims
    statistics(mfcc, 'mfcc', columns_name, data)  # stats = 20 dims * 7 stats data
    
    rms = librosa.feature.rms(x, sr) # rms with 1 raw dims
    statistics(rms, 'rms', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_centroid = librosa.feature.spectral_centroid(x , sr) # spectral_centroid with 1 raw dims
    statistics(spectral_centroid, 'spectral_centroid', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_bandwidth = librosa.feature.spectral_bandwidth(x , sr) # spectral_bandwidth with 1 raw dims
    statistics(spectral_bandwidth, 'spectral_bandwidth', columns_name, data) # stats = 1 dims * 7 stats data

    spectral_contrast = librosa.feature.spectral_contrast(x , sr) # spectral_contrast with 7 raw dims
    statistics(spectral_contrast, 'spectral_contrast', columns_name, data) # stats = 7 dims * 7 stats data
    
    spectral_rolloff = librosa.feature.spectral_rolloff(x , sr) # spectral_rolloff with 1 raw dims
    statistics(spectral_rolloff, 'spectral_rolloff', columns_name, data) # stats = 1 dims * 7 stats data

    tonnetz = librosa.feature.tonnetz(x , sr) # tonnetz with 6 raw dims
    statistics(tonnetz, 'tonnetz', columns_name, data) # stats = 6 dims * 7 stats data

    zero_crossing_rate = librosa.feature.zero_crossing_rate(x , sr) # zero_crossing_rate with 1 raw dims
    statistics(zero_crossing_rate, 'zero_crossing_rate', columns_name, data) # stats = 1 dims * 7 stats data

    # total number of expected dimensions - 518

    return data, columns_name

In [18]:
# extract audio freatures for all songs chosuses/Hook
i = 0
d, cols = extract_features( titles[i])
data.append(d)
print(f'The {i} song Done...')
i += 1

NameError: name 'paths' is not defined

In [12]:
newData = pd.DataFrame(data, columns=cols) # create dataframe for new extracted audio features 

In [13]:
newData

,title,chroma_stft_kew_0,chroma_stft_min_0,chroma_stft_max_0,chroma_stft_std_0,chroma_stft_mean_0,chroma_stft_median_0,chroma_stft_kurtosis_0,chroma_stft_kew_1,chroma_stft_min_1,...,tonnetz_mean_5,tonnetz_median_5,tonnetz_kurtosis_5,zero_crossing_rate_kew_0,zero_crossing_rate_min_0,zero_crossing_rate_max_0,zero_crossing_rate_std_0,zero_crossing_rate_mean_0,zero_crossing_rate_median_0,zero_crossing_rate_kurtosis_0
0,Blinding Lights,0.415558,0.013187,1.0,0.324938,0.478119,0.389486,-1.232214,0.987510,0.012746,...,0.007117,-0.001232,-0.514158,-0.230018,0.040272,0.125079,0.014692,0.088559,0.088163,0.301464
1,Good 4 U,1.029629,0.006761,1.0,0.271967,0.350076,0.263252,0.156509,0.371548,0.004671,...,-0.011907,-0.009472,0.097318,-0.282520,0.057007,0.229070,0.035503,0.148131,0.157211,-0.304652
2,Drivers License,1.395750,0.000648,1.0,0.307888,0.236975,0.087020,0.739643,1.646941,0.000456,...,0.001545,-0.009260,0.360876,0.742759,0.023492,0.178957,0.032571,0.077563,0.066463,-0.180427
3,Montero (Call Me By Your Name),0.926617,0.003467,1.0,0.242382,0.301311,0.262466,0.285644,0.843115,0.003586,...,-0.014834,-0.023444,0.456363,0.524002,0.051111,0.277596,0.045581,0.147997,0.142472,-0.344706
4,Butter,0.306467,0.038579,1.0,0.334490,0.521375,0.440152,-1.435872,0.876648,0.040661,...,0.005865,0.003965,-0.428400,2.549234,0.040590,0.225442,0.027880,0.097319,0.091950,7.586434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,Crash,0.077936,0.060204,1.0,0.268189,0.554176,0.542225,-1.177267,-0.139061,0.052480,...,-0.028667,-0.028262,0.448644,1.368431,0.085714,0.353741,0.045850,0.185233,0.173379,1.960386
766,Luxurious,-0.020788,0.013595,1.0,0.327147,0.570372,0.553099,-1.405522,1.232144,0.013089,...,0.002858,0.007586,0.429634,1.686463,0.045261,0.415374,0.070371,0.156106,0.137642,3.006131
767,If It's Lovin' That You Want,0.560997,0.000398,1.0,0.302554,0.458398,0.369975,-0.900844,0.880602,0.008680,...,0.004888,0.000771,-0.075453,0.344456,0.025941,0.214603,0.035465,0.114828,0.112834,0.024774
768,Unbreakable,0.412242,0.028240,1.0,0.291556,0.489242,0.398040,-1.180265,0.640337,0.016811,...,0.017730,-0.001053,1.118338,0.823857,0.010113,0.067619,0.012221,0.029605,0.029252,0.587163


In [16]:
df # previous data table

,Artist,Title,Label,song_path,extracted_chorus_path
0,The Weeknd,Blinding Lights,1,popular/Blinding Lights.mp3,chorus_extract/Blinding Lights.wav
1,Olivia Rodrigo,Good 4 U,1,popular/Good 4 U.mp3,chorus_extract/Good 4 U.wav
2,Olivia Rodrigo,Drivers License,1,popular/Drivers License.mp3,chorus_extract/Drivers License.wav
3,Lil Nas X,Montero (Call Me By Your Name),1,popular/Montero (Call Me By Your Name).mp3,chorus_extract/Montero (Call Me By Your Name).wav
4,BTS,Butter,1,popular/Butter.mp3,chorus_extract/Butter.wav
...,...,...,...,...,...
765,Gwen Stefani,Crash,0,unpopular/Crash.mp3,chorus_extract/Crash.wav
766,Gwen Stefani,Luxurious,0,unpopular/Luxurious.mp3,chorus_extract/Luxurious.wav
767,Rihanna,If It's Lovin' That You Want,0,unpopular/If It's Lovin' That You Want.mp3,chorus_extract/If It's Lovin' That You Want.wav
768,Alicia Keys,Unbreakable,0,unpopular/Unbreakable.mp3,chorus_extract/Unbreakable.wav


In [19]:
final_data = df.join(newData) # join both datasets of same number of rows 
final_data

,Artist,Title,Label,song_path,extracted_chorus_path,title,chroma_stft_kew_0,chroma_stft_min_0,chroma_stft_max_0,chroma_stft_std_0,...,tonnetz_mean_5,tonnetz_median_5,tonnetz_kurtosis_5,zero_crossing_rate_kew_0,zero_crossing_rate_min_0,zero_crossing_rate_max_0,zero_crossing_rate_std_0,zero_crossing_rate_mean_0,zero_crossing_rate_median_0,zero_crossing_rate_kurtosis_0
0,The Weeknd,Blinding Lights,1,popular/Blinding Lights.mp3,chorus_extract/Blinding Lights.wav,Blinding Lights,0.415558,0.013187,1.0,0.324938,...,0.007117,-0.001232,-0.514158,-0.230018,0.040272,0.125079,0.014692,0.088559,0.088163,0.301464
1,Olivia Rodrigo,Good 4 U,1,popular/Good 4 U.mp3,chorus_extract/Good 4 U.wav,Good 4 U,1.029629,0.006761,1.0,0.271967,...,-0.011907,-0.009472,0.097318,-0.282520,0.057007,0.229070,0.035503,0.148131,0.157211,-0.304652
2,Olivia Rodrigo,Drivers License,1,popular/Drivers License.mp3,chorus_extract/Drivers License.wav,Drivers License,1.395750,0.000648,1.0,0.307888,...,0.001545,-0.009260,0.360876,0.742759,0.023492,0.178957,0.032571,0.077563,0.066463,-0.180427
3,Lil Nas X,Montero (Call Me By Your Name),1,popular/Montero (Call Me By Your Name).mp3,chorus_extract/Montero (Call Me By Your Name).wav,Montero (Call Me By Your Name),0.926617,0.003467,1.0,0.242382,...,-0.014834,-0.023444,0.456363,0.524002,0.051111,0.277596,0.045581,0.147997,0.142472,-0.344706
4,BTS,Butter,1,popular/Butter.mp3,chorus_extract/Butter.wav,Butter,0.306467,0.038579,1.0,0.334490,...,0.005865,0.003965,-0.428400,2.549234,0.040590,0.225442,0.027880,0.097319,0.091950,7.586434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,Gwen Stefani,Crash,0,unpopular/Crash.mp3,chorus_extract/Crash.wav,Crash,0.077936,0.060204,1.0,0.268189,...,-0.028667,-0.028262,0.448644,1.368431,0.085714,0.353741,0.045850,0.185233,0.173379,1.960386
766,Gwen Stefani,Luxurious,0,unpopular/Luxurious.mp3,chorus_extract/Luxurious.wav,Luxurious,-0.020788,0.013595,1.0,0.327147,...,0.002858,0.007586,0.429634,1.686463,0.045261,0.415374,0.070371,0.156106,0.137642,3.006131
767,Rihanna,If It's Lovin' That You Want,0,unpopular/If It's Lovin' That You Want.mp3,chorus_extract/If It's Lovin' That You Want.wav,If It's Lovin' That You Want,0.560997,0.000398,1.0,0.302554,...,0.004888,0.000771,-0.075453,0.344456,0.025941,0.214603,0.035465,0.114828,0.112834,0.024774
768,Alicia Keys,Unbreakable,0,unpopular/Unbreakable.mp3,chorus_extract/Unbreakable.wav,Unbreakable,0.412242,0.028240,1.0,0.291556,...,0.017730,-0.001053,1.118338,0.823857,0.010113,0.067619,0.012221,0.029605,0.029252,0.587163


In [20]:
final_data.columns # scan for repeated column names 

Index(['Artist', 'Title', 'Label', 'song_path', 'extracted_chorus_path',
       'title', 'chroma_stft_kew_0', 'chroma_stft_min_0', 'chroma_stft_max_0',
       'chroma_stft_std_0',
       ...
       'tonnetz_mean_5', 'tonnetz_median_5', 'tonnetz_kurtosis_5',
       'zero_crossing_rate_kew_0', 'zero_crossing_rate_min_0',
       'zero_crossing_rate_max_0', 'zero_crossing_rate_std_0',
       'zero_crossing_rate_mean_0', 'zero_crossing_rate_median_0',
       'zero_crossing_rate_kurtosis_0'],
      dtype='object', length=524)

In [25]:
final_data = final_data.drop(columns=['title'], axis=1) #drop repeated colunm name

In [26]:
final_data.columns

Index(['Artist', 'Title', 'Label', 'song_path', 'extracted_chorus_path',
       'chroma_stft_kew_0', 'chroma_stft_min_0', 'chroma_stft_max_0',
       'chroma_stft_std_0', 'chroma_stft_mean_0',
       ...
       'tonnetz_mean_5', 'tonnetz_median_5', 'tonnetz_kurtosis_5',
       'zero_crossing_rate_kew_0', 'zero_crossing_rate_min_0',
       'zero_crossing_rate_max_0', 'zero_crossing_rate_std_0',
       'zero_crossing_rate_mean_0', 'zero_crossing_rate_median_0',
       'zero_crossing_rate_kurtosis_0'],
      dtype='object', length=523)

In [27]:
#save table to csv file 
final_data.to_csv('feature_extract.csv', index=False) 